In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [2]:
teams_url = 'http://www.espn.com/nba/teams'
players_url = ''

In [3]:
html = urllib.request.urlopen(teams_url)
soup = BeautifulSoup(html, 'lxml')

In [4]:
links_store = str(soup.find_all('a'))

pattern = re.compile(r'href="/nba/team/_/name/\w*/\w*[-]?\w*')
matches = pattern.finditer(links_store)

teams = set()
for match in matches: 
    teams.add(match.group(0))

abbr_name = []
for link in teams:
    temp = link.split('/')
    abbr_name.append([temp[5], temp[6]])

In [8]:
# # scrape the data for home games
# game_type = 33
master_df = pd.DataFrame(columns=['PNAME', 'POS', 'TEAM',  'YEAR', 'TYPE', 'GP', 'GS', 'MIN', 'PTS', 'OR', 'DR', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'AST/TO', 'PER'])
year_list = range(2002, 2021)
for item in abbr_name:
    team_abbr = item[0]
    for year in year_list:
        season_no = year
        season_type = '2'
        game_types = ['33', '34']
        for game_type in game_types:
            time.sleep(3)
            test_split_link = 'https://www.espn.in/nba/team/stats/_/name/{team_abbr}/season/{season_no}/seasontype/{season_type}/split/{game_type}'.format(team_abbr=team_abbr, season_no=season_no, season_type=season_type, game_type=game_type)
            print(test_split_link)
            test_html = urllib.request.urlopen(test_split_link)
            test_soup = BeautifulSoup(test_html, 'lxml')
            players = []
            try:
                tables = test_soup.find_all('table')
                player_names = tables[0]
                rows = player_names.find_all('tr')
                # ignore first row
                for row in rows[1:]:
                    tds = row.find_all('td')[0]
                    spans = tds.find_all('span')[0]
                    try:
                        temp = [span.getText() for span in spans]
                        # remove trailing '*' after some of the position values
                        if len(temp) > 2:
                            temp = temp[:-1]
                        # removing leading spaces for position
                        temp[1] = temp[1].lstrip(' ')
                        players.append(temp)
                    except Exception as e:
                        pass

                all_rows = []
                tables = test_soup.find_all('table')
                player_stats = tables[1]
                rows = player_stats.find_all('tr')
                for row in rows[1:-1]:
                    tds = row.find_all('td')
                    vals = [td.getText() for td in tds]
                    all_rows.append(vals)

                concat_name_stats = []
                for idx in range(len(players)):
                    concat_name_stats.append(players[idx] + [team_abbr] + [season_no] + [game_type] +  all_rows[idx])

                temp_df = pd.DataFrame(concat_name_stats,columns=['PNAME', 'POS', 'TEAM',  'YEAR', 'TYPE', 'GP', 'GS', 'MIN', 'PTS', 'OR', 'DR', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'AST/TO', 'PER'])
                master_df = pd.concat([master_df, temp_df], axis=0)
            except Exception as e:
                pass

https://www.espn.in/nba/team/stats/_/name/mil/season/2020/seasontype/2/split/33


[['11.5',
  '20.5',
  '56.0',
  '1.5',
  '5.0',
  '30.7',
  '6.4',
  '10.7',
  '59.7',
  '10.0',
  '15.5',
  '64.2',
  '1.505',
  '0.60'],
 ['7.4',
  '14.5',
  '51.4',
  '2.4',
  '5.5',
  '43.6',
  '3.1',
  '3.5',
  '90.4',
  '5.0',
  '9.0',
  '56.1',
  '1.410',
  '0.60'],
 ['5.1',
  '10.7',
  '48.2',
  '1.2',
  '3.1',
  '39.6',
  '2.6',
  '3.4',
  '75.0',
  '3.9',
  '7.5',
  '51.8',
  '1.324',
  '0.54'],
 ['3.6',
  '9.3',
  '38.4',
  '1.3',
  '4.9',
  '26.9',
  '1.7',
  '1.9',
  '88.3',
  '2.3',
  '4.4',
  '51.1',
  '1.088',
  '0.45'],
 ['3.4',
  '6.1',
  '54.7',
  '1.3',
  '2.9',
  '45.7',
  '1.6',
  '2.0',
  '80.0',
  '2.0',
  '3.3',
  '62.6',
  '1.564',
  '0.65'],
 ['2.9',
  '6.5',
  '44.4',
  '2.0',
  '4.7',
  '41.7',
  '1.2',
  '1.6',
  '77.4',
  '0.9',
  '1.8',
  '51.7',
  '1.380',
  '0.59'],
 ['3.3',
  '7.7',
  '43.5',
  '1.2',
  '3.6',
  '31.9',
  '1.0',
  '1.2',
  '82.1',
  '2.2',
  '4.1',
  '53.8',
  '1.150',
  '0.51'],
 ['2.7',
  '5.6',
  '47.2',
  '1.1',
  '2.4',
  '46.1',

In [ ]:
master_df
master_df.to_csv('home_away.csv', index=False)